In [2]:
import pandas as pd
from functools import reduce
from cyvcf2 import VCF

In [3]:
#### Compare with previous overlaps found with bedtools where TR shouldn't be entirely inside coding region ####

old_overlaps = pd.read_csv("/expanse/projects/gymreklab/helia/ensembl/experiments/coding_regions/mapping.txt", sep = "\t", header = None)

old_overlaps_entire = []
for index, row in old_overlaps.iterrows():
    if row[1] >= row[4] and row[2] <= row[5]: # check for entirely being inside the gene
        old_overlaps_entire.append(row)
old_overlaps_entire = pd.DataFrame(old_overlaps_entire)
old_overlaps_entire.columns =  ['chrom','TR_start', 'TR_end','chrom2', 'gene_start', 'gene_end','Gene_name','y','z']
old_overlaps_entire = old_overlaps_entire.drop_duplicates(subset = ['chrom','TR_start', 'TR_end'])
old_overlaps_entire[['TR_start', 'TR_end', 'gene_start', 'gene_end']] = old_overlaps_entire[['TR_start', 'TR_end', 'gene_start', 'gene_end']].astype(int)
old_overlaps_entire

      
      

,chrom,TR_start,TR_end,chrom2,gene_start,gene_end,Gene_name,y,z
9,chr1,960770,960798,chr1,960693,960800,ENST00000338591.8_cds_0_0_chr1_960694_f,0,+
10,chr1,970841,970857,chr1,970685,971006,ENST00000379409.6_cds_4_0_chr1_970686_f,0,+
11,chr1,976178,976199,chr1,976171,976269,ENST00000433179.4_cds_3_0_chr1_976172_r,0,-
14,chr1,1232633,1232644,chr1,1232278,1233268,ENST00000379198.5_cds_0_0_chr1_1232279_f,0,+
15,chr1,1232807,1232818,chr1,1232278,1233268,ENST00000379198.5_cds_0_0_chr1_1232279_f,0,+
...,...,...,...,...,...,...,...,...,...
20831,chr22,50603316,50603361,chr22,50603222,50603498,ENST00000329492.6_cds_2_0_chr22_50603223_f,0,+
20832,chr22,50604149,50604160,chr22,50603840,50605064,ENST00000329492.6_cds_4_0_chr22_50603841_f,0,+
20833,chr22,50604544,50604573,chr22,50603840,50605064,ENST00000329492.6_cds_4_0_chr22_50603841_f,0,+
20834,chr22,50604546,50604557,chr22,50603840,50605064,ENST00000329492.6_cds_4_0_chr22_50603841_f,0,+


In [10]:
old_overlaps_entire[['chrom','TR_start','TR_end']].to_csv("TR_intersect.txt", sep = "\t", header=False, index = False)

In [9]:
#### Reading heterozygousity outputs ####

AFR_df = pd.DataFrame(columns = ["chrom","start","end","het-1"])
AMR_df = pd.DataFrame(columns = ["chrom","start","end","het-1"])
EAS_df = pd.DataFrame(columns = ["chrom","start","end","het-1"])
SAS_df = pd.DataFrame(columns = ["chrom","start","end","het-1"])
EUR_df = pd.DataFrame(columns = ["chrom","start","end","het-1"])

all_df = [EUR_df, SAS_df, EAS_df, AMR_df, AFR_df]


    

pops = ['EUR', 'SAS', 'EAS', 'AMR', 'AFR']
for i in range(5):
    for chrom in range(1,23):
        df = pd.read_csv(f"/expanse/projects/gymreklab/helia/ensembl/experiments/allele_freq/freqs/freqs_chr{chrom}_{pops[i]}.tab",
                         sep="\t", usecols=['chrom', 'start', 'end', 'het-1'])
        
        df = df.drop_duplicates(subset = ["chrom", "start","end"])
        df['end'] = df['end'] - 1
        df = pd.merge(df, old_overlaps_entire, left_on=["chrom", "start","end"], 
                      right_on = ["chrom", "TR_start","TR_end"])
        df = df[['chrom', 'start', 'end', "Gene_name", 'het-1']]
        all_df[i] = pd.concat([all_df[i], df])

    

In [17]:
### Merging all pops ####

df_all_pop = reduce(lambda  left,right: pd.merge(left,right,on=['chrom','start','end', 'Gene_name'],
                                            how='inner'), all_df)
df_all_pop.columns = ['chrom', 'start', 'end', 'EUR_het', 'gene_name',
                      'SAS_het', 'EAS_het', 'AMR_het', 'AFR_het']
df_all_pop = df_all_pop[['chrom', 'start', 'end', 'gene_name', 'EUR_het',
                      'SAS_het', 'EAS_het', 'AMR_het', 'AFR_het']]
df_all_pop

,chrom,start,end,gene_name,EUR_het,SAS_het,EAS_het,AMR_het,AFR_het
0,chr1,960770,960798,ENST00000338591.8_cds_0_0_chr1_960694_f,0.00321,0.00000,0.00000,0.0,0.00233
1,chr1,970841,970857,ENST00000379409.6_cds_4_0_chr1_970686_f,0.00000,0.00000,0.00000,0.0,0.00000
2,chr1,976178,976199,ENST00000433179.4_cds_3_0_chr1_976172_r,0.00000,0.00332,0.00000,0.0,0.00000
3,chr1,1232633,1232644,ENST00000379198.5_cds_0_0_chr1_1232279_f,0.00000,0.00000,0.00000,0.0,0.00000
4,chr1,1232807,1232818,ENST00000379198.5_cds_0_0_chr1_1232279_f,0.00000,0.00000,0.00000,0.0,0.00000
...,...,...,...,...,...,...,...,...,...
6069,chr22,50603316,50603361,ENST00000329492.6_cds_2_0_chr22_50603223_f,0.00000,0.00000,0.00341,0.0,0.00447
6070,chr22,50604149,50604160,ENST00000329492.6_cds_4_0_chr22_50603841_f,0.00000,0.00000,0.00000,0.0,0.00000
6071,chr22,50604544,50604573,ENST00000329492.6_cds_4_0_chr22_50603841_f,0.00000,0.00000,0.00000,0.0,0.00000
6072,chr22,50604546,50604557,ENST00000329492.6_cds_4_0_chr22_50603841_f,0.00000,0.00000,0.00000,0.0,0.00000


In [20]:
#### Load Motif and polymorphism ####
motifs = []
for chrom in range(1,23):
    vcf = VCF(f"/expanse/projects/gymreklab/helia/ensembl/experiments/coding_regions/heterozygosity/coding_calls_{chrom}.vcf.gz")
    for record in vcf:
        motifs.append([record.CHROM, record.POS, record.INFO['END'], record.INFO['RU'], record.ALT])

motifs = pd.DataFrame(motifs, columns = ['chrom', 'start', 'end', 'motif', 'alt'])
motifs = motifs.drop_duplicates(subset = ["chrom", "start","end"])
df_all_pop_motif = pd.merge(df_all_pop, motifs, on = ['chrom', 'start', 'end'])
display(df_all_pop_motif)
polymorphism_percentage = len(df_all_pop_motif[df_all_pop_motif['alt'].map(len) != 0])/len(df_all_pop_motif)
print(f"Polymorphism percentage in coding regions: polymorphic_cnt: {polymorphism_percentage}")
df_all_pop_motif.drop(labels = "alt", axis = 1).to_csv("tables/Supplementary_Table_7.csv", index = False, sep = "\t")

,chrom,start,end,gene_name,EUR_het,SAS_het,EAS_het,AMR_het,AFR_het,motif,alt
0,chr1,960770,960798,ENST00000338591.8_cds_0_0_chr1_960694_f,0.00321,0.00000,0.00000,0.0,0.00233,CCG,[CGCCGCCG]
1,chr1,970841,970857,ENST00000379409.6_cds_4_0_chr1_970686_f,0.00000,0.00000,0.00000,0.0,0.00000,AC,[]
2,chr1,976178,976199,ENST00000433179.4_cds_3_0_chr1_976172_r,0.00000,0.00332,0.00000,0.0,0.00000,AGCCCC,"[GCTGGGGCTGGGGCTGGGGCTG, GCTGGGGCTGGGGCTGTGGCT..."
3,chr1,1232633,1232644,ENST00000379198.5_cds_0_0_chr1_1232279_f,0.00000,0.00000,0.00000,0.0,0.00000,AGC,[]
4,chr1,1232807,1232818,ENST00000379198.5_cds_0_0_chr1_1232279_f,0.00000,0.00000,0.00000,0.0,0.00000,CCG,"[CGCCGC, CGCCGCCGC, CGC]"
...,...,...,...,...,...,...,...,...,...,...,...
6069,chr22,50603316,50603361,ENST00000329492.6_cds_2_0_chr22_50603223_f,0.00000,0.00000,0.00341,0.0,0.00447,AGG,"[GAGGAGGACGATGAGGACGAGGAAGAGGAGGAGGAGGAGGAGG, ..."
6070,chr22,50604149,50604160,ENST00000329492.6_cds_4_0_chr22_50603841_f,0.00000,0.00000,0.00000,0.0,0.00000,AGC,[AGC]
6071,chr22,50604544,50604573,ENST00000329492.6_cds_4_0_chr22_50603841_f,0.00000,0.00000,0.00000,0.0,0.00000,CCGCG,"[CGCGCCGCCGCGGCCCGCGCCCGCCGCGCC, CGCGCCGCCGCCG..."
6072,chr22,50604546,50604557,ENST00000329492.6_cds_4_0_chr22_50603841_f,0.00000,0.00000,0.00000,0.0,0.00000,CCG,[]


Polymorphism percentage in coding regions: polymorphic_cnt: 0.5408297662166612


In [21]:
#### Polymorphims in whole genome ####
all_records = 0
polymorphic = 0
for i in range(1,23):
    df = pd.read_csv(f"/expanse/projects/gymreklab/helia/ensembl/experiments/coding_regions/info/stats_chr{i}.txt",
                     sep = "\t", header = None)    
    all_records += len(df)
    polymorphic += len(df[df[4] != "."])
    
print(all_records, polymorphic, polymorphic/all_records)

1440104 1123154 0.7799117285973791
